In [4]:
%pip install fastapi uvicorn wikipedia reportlab nest_asyncio



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from fastapi import FastAPI, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os
import uuid
import nest_asyncio
from urllib.parse import quote_plus

nest_asyncio.apply()
app = FastAPI()

# CORS setup to allow frontend access
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173", "http://127.0.0.1:5173"], 
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
def welcome():
    return {"message": "API is running!"}

def search_wikipedia(query: str, level: str):
    try:
        page = wikipedia.page(query)
        summary = page.summary
        wiki_url = page.url

        sentences = summary.split(". ")
        beginner = ". ".join(sentences[:2]) + "."
        intermediate = ". ".join(sentences[:4]) + "."
        advanced = summary

        if level == "beginner":
            return beginner, wiki_url
        elif level == "intermediate":
            return intermediate, wiki_url
        elif level == "advanced":
            return advanced, wiki_url
        else:
            return "Invalid level. Choose beginner, intermediate, or advanced.", ""
    except DisambiguationError:
        return f"Multiple topics found for '{query}'. Please be more specific.", ""
    except PageError:
        return f"No Wikipedia page found for '{query}'.", ""
    except Exception as e:
        return f"An error occurred: {str(e)}", ""

def draw_wrapped_text(c, text, x, y, max_width, line_height):
    words = text.split()
    line = ""
    for word in words:
        test_line = line + word + " "
        if c.stringWidth(test_line, "Helvetica", 12) <= max_width:
            line = test_line
        else:
            c.drawString(x, y, line)
            y -= line_height
            line = word + " "
    if line:
        c.drawString(x, y, line)
        y -= line_height
    return y

def cleanup_file(path: str):
    if os.path.exists(path):
        os.remove(path)

@app.get("/download_pdf/")
def download_pdf(question: str, level: str = "beginner", background_tasks: BackgroundTasks = BackgroundTasks()):
    answer, wiki_url = search_wikipedia(question, level)
    if wiki_url == "":
        raise HTTPException(status_code=404, detail=answer)

    youtube_url = f"https://www.youtube.com/results?search_query={quote_plus(question)}+{level}"

    filename = f"{uuid.uuid4().hex}.pdf"
    filepath = os.path.join(".", filename)

    c = canvas.Canvas(filepath, pagesize=letter)
    width, height = letter
    y = height - 50

    c.setFont("Helvetica-Bold", 14)
    c.drawString(50, y, f"Question: {question}")
    y -= 25

    c.setFont("Helvetica", 12)
    c.drawString(50, y, f"Level: {level}")
    y -= 30

    c.drawString(50, y, "Answer:")
    y -= 20
    y = draw_wrapped_text(c, answer, 50, y, 500, 15)

    c.drawString(50, y, "Wikipedia Link:")
    y -= 15
    y = draw_wrapped_text(c, wiki_url, 50, y, 500, 15)

    c.drawString(50, y, "YouTube Link:")
    y -= 15
    y = draw_wrapped_text(c, youtube_url, 50, y, 500, 15)

    c.save()
    background_tasks.add_task(cleanup_file, filepath)

    return FileResponse(
        filepath,
        media_type="application/pdf",
        filename="summary.pdf",
        background=background_tasks
    )

@app.get("/ask/")
def ask_question(question: str, level: str = "beginner"):
    answer, wiki_url = search_wikipedia(question, level)
    encoded_q = quote_plus(question)

    return {
        "question": question,
        "level": level,
        "answer": answer,
        "videos": f"https://www.youtube.com/results?search_query={encoded_q}+{level}",
        "pdf_download_link": f"http://127.0.0.1:3000/download_pdf?question={encoded_q}&level={level}",
        "wikipedia_link": wiki_url
    }

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=3000)


INFO:     Started server process [17872]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:3000 (Press CTRL+C to quit)


INFO:     127.0.0.1:64928 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64929 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:64975 - "GET /ask?question=c%2B%2B&level=beginner HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:64975 - "GET /ask/?question=c%2B%2B&level=beginner HTTP/1.1" 200 OK
INFO:     127.0.0.1:64985 - "GET /download_pdf?question=c%2B%2B&level=beginner HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:64985 - "GET /download_pdf/?question=c%2B%2B&level=beginner HTTP/1.1" 200 OK
